# Astra DB Serverless, Python Quickstart

Learn how to create a [Serverless (Vector) database](https://docs.datastax.com/en/astra-db-serverless/get-started/astra-db-introduction.html), connect to your database, load a set of vector embeddings, and perform a similarity search to find vectors that are close to the one in your query.

## Prerequisites

To run this notebook, you need an Astra DB "Serverless (Vector) database" and the connection parameters to access it:

- the *Application Token* (a string starting with `AstraCS:...`)
- the *API Endpoint* (example: `https://01234567-89ab-cdef-0123-456789abcdef-us-east1.apps.astra.datastax.com`)

Please consult the [documentation page for this quickstart](https://docs.datastax.com/en/astra-db-serverless/get-started/quickstart.html#create-a-serverless-vector-database) for more details on the above requirements.

## Install the client

In [ ]:
!pip install --quiet --upgrade astrapy

## Import needed libraries

In [ ]:
import json
import os
from getpass import getpass

from astrapy import DataAPIClient
from astrapy.constants import VectorMetric
from astrapy.ids import UUID
from astrapy.exceptions import InsertManyException

## Provide database credentials

In [ ]:
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("ASTRA_DB_APPLICATION_TOKEN = ")
os.environ["ASTRA_DB_API_ENDPOINT"] = input("ASTRA_DB_API_ENDPOINT = ")

## Work with your data

### Connect to the database

Initialize the client and get a "Database" object

In [ ]:
client = DataAPIClient(os.environ["ASTRA_DB_APPLICATION_TOKEN"])
database = client.get_database(os.environ["ASTRA_DB_API_ENDPOINT"])
print(f"* Database: {database.info().name}\n")

### Create a collection

Create a collection. The default similarity metric is cosine.

Choose dimensions that match your vector data.

If you're not sure, use the vector dimension that your embeddings model produces.

In [ ]:
collection = database.create_collection(
    "vector_test",
    dimension=5,
    metric=VectorMetric.COSINE,  # Or just 'cosine'.
    check_exists=False, # Optional
)
print(f"* Collection: {collection.full_name}\n")

### Insert documents with embeddings

Insert documents with embeddings into the collection.

UUIDs are version 7.

In [ ]:
documents = [
    {
        "_id": UUID("018e65c9-df45-7913-89f8-175f28bd7f74"),
        "text": "Chat bot integrated sneakers that talk to you",
        "$vector": [0.1, 0.15, 0.3, 0.12, 0.05],
    },
    {
        "_id": UUID("018e65c9-e1b7-7048-a593-db452be1e4c2"),
        "text": "An AI quilt to help you sleep forever",
        "$vector": [0.45, 0.09, 0.01, 0.2, 0.11],
    },
    {
        "_id": UUID("018e65c9-e33d-749b-9386-e848739582f0"),
        "text": "A deep learning display that controls your mood",
        "$vector": [0.1, 0.05, 0.08, 0.3, 0.6],
    },
]

In [ ]:
try:
    insertion_result = collection.insert_many(documents)
    print(f"* Inserted {len(insertion_result.inserted_ids)} items.\n")
except InsertManyException:
    print("* Documents found on DB already. Let's move on.\n")

### Perform a similarity search

In [ ]:
query_vector = [0.15, 0.1, 0.1, 0.35, 0.55]
results = collection.find(
    sort={"$vector": query_vector},
    limit=10,
)
print("Vector search results:")
for document in results:
    print("    ", document)

## (optional) Cleanup

In [ ]:
drop_result = collection.drop()
print(drop_result)

## Conclusion

This has just scratched the surface of what you can do.

To find out how easy it is to build your GenAI application with Astra DB, take a look at [the documentation](https://docs.datastax.com/en/astra-db-serverless/index.html).